In [1]:
from IPython.core.display import display, HTML
display(HTML("<style>.container { width:90% !important; }</style>"))

In [2]:
import pandas as pd
import numpy as np
import datetime as DT

In [3]:
postulant_gen_age_original = pd.read_csv("../../datos_navent_fiuba/fiuba_2_postulantes_genero_y_edad.csv")
postulant_gen_age_from_15 = pd.read_csv("../../fiuba_2_postulantes_genero_y_edad.csv")
postulant_gen_age_to_15 = pd.read_csv("../../entrega6/fiuba_2_postulantes_genero_y_edad.csv")

### Veo como son los datos y la longitud de los csv

In [4]:
postulant_gen_age_original.head()

,idpostulante,fechanacimiento,sexo
0,NM5M,1970-12-03,FEM
1,5awk,1962-12-04,FEM
2,ZaO5,1978-08-10,FEM
3,NdJl,1969-05-09,MASC
4,eo2p,1981-02-16,MASC


In [5]:
print(len(postulant_gen_age_original))

200888


In [6]:
postulant_gen_age_from_15.head()

,idpostulante,fechanacimiento,sexo
0,6MM,1985-01-01,MASC
1,ZX1,NaN,NO_DECLARA
2,YZP,1961-05-01,MASC
3,ebE,1952-07-07,MASC
4,N1x,NaN,NO_DECLARA


In [7]:
print(len(postulant_gen_age_from_15))

281385


In [8]:
postulant_gen_age_to_15.head()

,idpostulante,fechanacimiento,sexo
0,6MM,1985-01-01,MASC
1,Nzz,NaN,NO_DECLARA
2,ZX1,NaN,NO_DECLARA
3,Nq5,NaN,NO_DECLARA
4,ebE,1952-07-07,MASC


In [9]:
print(len(postulant_gen_age_to_15))

297747


### Mergeo los primeros 2 csv

In [10]:
pos_aux = pd.merge(postulant_gen_age_original, postulant_gen_age_from_15, on = ['idpostulante', 'fechanacimiento', 'sexo'], how = 'outer')
pos_aux.head()

,idpostulante,fechanacimiento,sexo
0,NM5M,1970-12-03,FEM
1,5awk,1962-12-04,FEM
2,ZaO5,1978-08-10,FEM
3,NdJl,1969-05-09,MASC
4,eo2p,1981-02-16,MASC


In [11]:
print(len(pos_aux))

392934


### Mergeo el csv mergeado anterior con el último que faltaba

In [12]:
postulant_gen_age = pd.merge(pos_aux, postulant_gen_age_to_15, on = ['idpostulante', 'fechanacimiento', 'sexo'], how = 'outer')
postulant_gen_age.head()

,idpostulante,fechanacimiento,sexo
0,NM5M,1970-12-03,FEM
1,5awk,1962-12-04,FEM
2,ZaO5,1978-08-10,FEM
3,NdJl,1969-05-09,MASC
4,eo2p,1981-02-16,MASC


In [13]:
print(len(postulant_gen_age))

505382


### Elimino registros de personas que no hayan declarado su edad y/o sexo y además los registros en los que la fecha de nacimiento es errónea.

In [14]:
#Elimino registros donde la fecha de nacimiento es errónea o ya es NaN
postulant_gen_age['fechanacimiento'] = pd.to_datetime(postulant_gen_age['fechanacimiento'], errors='coerce')
postulant_gen_age = postulant_gen_age.dropna(subset=['fechanacimiento'])
print(len(postulant_gen_age))

478664


In [15]:
#Elimino registros donde no se declara el sexo
postulant_gen_age = postulant_gen_age[postulant_gen_age['sexo'] != 'NO_DECLARA']
print(len(postulant_gen_age))

477554


En total se eliminaron 27.828 registros, esto es el 5,5063 % de los datos originales (es decir, de los 505382). Como no es mucho, los desprecio.

### Agrego una columna con la edad de la persona

In [16]:
now = pd.Timestamp(DT.datetime.now())
postulant_gen_age['fechanacimiento'] = postulant_gen_age['fechanacimiento'].where(postulant_gen_age['fechanacimiento'] < now, postulant_gen_age['fechanacimiento'] -  np.timedelta64(100, 'Y'))
postulant_gen_age['edad'] = (now - postulant_gen_age['fechanacimiento']).astype('<m8[Y]')
postulant_gen_age['edad'] = postulant_gen_age['edad'].apply(lambda x: int(x))
postulant_gen_age.head()

,idpostulante,fechanacimiento,sexo,edad
0,NM5M,1970-12-03,FEM,47
1,5awk,1962-12-04,FEM,55
2,ZaO5,1978-08-10,FEM,39
3,NdJl,1969-05-09,MASC,49
4,eo2p,1981-02-16,MASC,37


In [17]:
#Reviso que no se haya modificado la cantidad de datos
print(len(postulant_gen_age))

477554


In [18]:
postulant_gen_age.drop('fechanacimiento', 1, inplace = True)

In [19]:
postulant_gen_age.head()

,idpostulante,sexo,edad
0,NM5M,FEM,47
1,5awk,FEM,55
2,ZaO5,FEM,39
3,NdJl,MASC,49
4,eo2p,MASC,37


### Lo guardo en un CSV limpio

In [20]:
postulant_gen_age.to_csv("../csvs/fiuba_2_postulantes_genero_y_edad.csv", index=False)